In [ ]:
import tensorflow as tf

print(tf.__version__)

2.4.1


In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/messy-vs-clean-room.zip \
  -O /tmp/messy_vs_clean_room.zip

--2021-05-07 23:08:06--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/messy-vs-clean-room.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70392746 (67M) [application/zip]
Saving to: ‘/tmp/messy_vs_clean_room.zip’

/tmp/messy_vs_clean 100%[===================>]  67.13M  1.46MB/s    in 57s     

2021-05-07 23:09:03 (1.18 MB/s) - ‘/tmp/messy_vs_clean_room.zip’ saved [70392746/70392746]



In [ ]:
import zipfile, os

local_zip = '/tmp/messy_vs_clean_room.zip'
zip_ref = zipfile.ZipFile(local_zip, mode='r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
os.listdir('/tmp/images/train')

['messy', 'clean']

In [ ]:
os.listdir('/tmp/images/val')

['messy', 'clean']

In [ ]:
train_clean_dir = os.path.join(train_dir, 'clean')
train_messy_dir = os.path.join(train_dir, 'messy')

validation_clean_dir = os.path.join(validation_dir, 'clean')
validation_messy_dir = os.path.join(validation_dir, 'messy')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   horizontal_flip=True, 
                                   shear_range=0.2, 
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   horizontal_flip=True, 
                                   shear_range=0.2, 
                                   fill_mode='nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150), 
                                                    batch_size=4, 
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir, 
                                                    target_size=(150, 150), 
                                                    batch_size=4, 
                                                    class_mode='binary')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
layers = tf.keras.layers

model = tf.keras.Sequential([
  layers.Conv2D(32, (3,3), activation='relu',input_shape=(150,150,3)),
  layers.MaxPooling2D(),
  layers.Conv2D(64, (3,3), activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, (3,3), activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, (3,3), activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=tf.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
model.fit(train_generator, 
          steps_per_epoch=25, 
          epochs=20, 
          validation_data=validation_generator, 
          validation_steps=5, 
          verbose=2)

Epoch 1/20
25/25 - 8s - loss: 0.7649 - accuracy: 0.5500 - val_loss: 0.6787 - val_accuracy: 0.5000
Epoch 2/20
25/25 - 7s - loss: 0.6859 - accuracy: 0.5200 - val_loss: 0.6615 - val_accuracy: 0.8000
Epoch 3/20
25/25 - 7s - loss: 0.6051 - accuracy: 0.7500 - val_loss: 0.8559 - val_accuracy: 0.5500
Epoch 4/20
25/25 - 7s - loss: 0.6385 - accuracy: 0.6500 - val_loss: 0.4955 - val_accuracy: 0.8000
Epoch 5/20
25/25 - 7s - loss: 0.6026 - accuracy: 0.6700 - val_loss: 0.5435 - val_accuracy: 0.8000
Epoch 6/20
25/25 - 7s - loss: 0.6007 - accuracy: 0.6700 - val_loss: 0.5741 - val_accuracy: 0.7000
Epoch 7/20
25/25 - 7s - loss: 0.5891 - accuracy: 0.6800 - val_loss: 0.6330 - val_accuracy: 0.5500
Epoch 8/20
25/25 - 7s - loss: 0.6520 - accuracy: 0.5700 - val_loss: 0.5834 - val_accuracy: 0.7500
Epoch 9/20
25/25 - 7s - loss: 0.5658 - accuracy: 0.7100 - val_loss: 0.5298 - val_accuracy: 0.7500
Epoch 10/20
25/25 - 7s - loss: 0.5603 - accuracy: 0.7500 - val_loss: 0.4833 - val_accuracy: 0.8000
Epoch 11/20
25/25 -

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():

  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)

  print(fn)
  if(classes == 0):
    print('clean')
  else:
    print('messy')